In [1]:
import numpy as np 
import pandas as pd 


import os 
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

./data/test.csv
./data/train.csv
./data/gender_submission.csv


In [52]:
o = pd.read_csv("./data/train.csv", index_col="PassengerId")
t = pd.read_csv("./data/test.csv", index_col="PassengerId")

In [41]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [5]:
train_data.index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            882, 883, 884, 885, 886, 887, 888, 889, 890, 891],
           dtype='int64', name='PassengerId', length=891)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

train_data.drop(["Name","Ticket"], axis=1, inplace=True)
test_data.drop(["Name","Ticket"], axis=1, inplace=True)
valid_train = train_data.drop(["Survived"], axis=1)

train_data['Cabin'] = train_data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test_data['Cabin'] = test_data["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

num_cols = [col for col in train_data.columns if train_data[col].dtype in ["int64", "float64"]]
tex_cols = [col for col in train_data.columns if train_data[col].dtype in ["object"]]
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,0,S
2,1,1,female,38.0,1,0,71.2833,1,C
3,1,3,female,26.0,0,0,7.9250,0,S
4,1,1,female,35.0,1,0,53.1000,1,S
5,0,3,male,35.0,0,0,8.0500,0,S


In [13]:
tex_cols

['Sex', 'Embarked']

In [16]:
one = OneHotEncoder(handle_unknown="ignore", sparse=False)
one_cols_train = pd.DataFrame(one.fit_transform(train_data[tex_cols]))
one_cols_valid = pd.DataFrame(one.fit_transform(test_data[tex_cols]))

In [20]:
one_cols_train.index = train_data.index
one_cols_valid.index = test_data.index

In [22]:
train_data = train_data.drop(tex_cols, axis=1)
test_data = test_data.drop(tex_cols, axis=1)

train_data = pd.concat([train_data, one_cols_train], axis=1)
test_data = pd.concat([test_data, one_cols_valid], axis=1)

In [23]:
train_data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,0,1,2,3,4,5
PassengerId,,,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1,38.0,1,0,71.2833,1,1.0,0.0,1.0,0.0,0.0,0.0
3,1,3,26.0,0,0,7.9250,0,1.0,0.0,0.0,0.0,1.0,0.0
4,1,1,35.0,1,0,53.1000,1,1.0,0.0,0.0,0.0,1.0,0.0
5,0,3,35.0,0,0,8.0500,0,0.0,1.0,0.0,0.0,1.0,0.0


In [71]:
"""y = lambda x: 0 if type(x) == float else 1
num_tran = ["Age"]
numerical_transformer = SimpleImputer(strategy="most_frequent")
pipeline1 = Pipeline(steps=[("imp", SimpleImputer(strategy="most_frequent")),
("one", OneHotEncoder(handle_unknown="ignore"))
])
preprocesor = ColumnTransformer([("cab", y, ["Cabin"]),
("num",numerical_transformer,num_cols),
("cat",pipeline1, tex_cols)])"""


In [73]:
"""from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
forest_model = RandomForestRegressor(random_state=1)
#forest_model = DecisionTreeClassifier(random_state=1)
my_pipeline = Pipeline(steps=[("preprocesor",preprocesor),
("model",forest_model)])
my_pipeline.fit(train_data,valid_train)
preds = my_pipeline.predict(test_data)"""


'from sklearn.ensemble import RandomForestRegressor\nfrom sklearn.tree import DecisionTreeClassifier\nforest_model = RandomForestRegressor(random_state=1)\n#forest_model = DecisionTreeClassifier(random_state=1)\nmy_pipeline = Pipeline(steps=[("preprocesor",preprocesor),\n("model",forest_model)])\nmy_pipeline.fit(train_data,valid_train)\npreds = my_pipeline.predict(test_data)'

In [49]:
prueba.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,0,S
2,1,1,female,38.0,1,0,71.2833,1,C
3,1,3,female,26.0,0,0,7.9250,0,S
4,1,1,female,35.0,1,0,53.1000,1,S
5,0,3,male,35.0,0,0,8.0500,0,S


In [62]:
y = lambda x: 0 if type(x) == float else 1
categorical_transformer = ColumnTransformer([("imp", y,"cabin")])

In [42]:
from sklearn.ensemble import RandomForestRegressor

imputer = SimpleImputer(strategy="most_frequent")
l = pd.DataFrame(imputer.fit_transform(train_data))
m = pd.DataFrame(imputer.fit_transform(test_data))

In [45]:
l.index = train_data.index
m.index = test_data.index

l.columns = train_data.columns
m.columns = test_data.columns

In [64]:
l.drop("Survived" ,axis=1, inplace=True)
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(l,valid_train)

RandomForestRegressor(random_state=0)

In [65]:
predicts = model.predict(m)

ValueError: X has 11 features, but DecisionTreeRegressor is expecting 12 features as input.

In [68]:
l.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 0, 1, 2, 3, 4, 5], dtype='object')

In [69]:
m.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 0, 1, 2, 3, 4], dtype='object')